# ML4FG Interim Report Classical Code
### By: Austin Stiefelmaier 11/11/23

## Imports and Settings

In [1]:
# General imports
import os
import copy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report


# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import TensorDataset

In [2]:
# Fix random num generation for reproducibility 
torch.manual_seed(7)
np.random.seed(7)
# os.environ["OMP_NUM_THREADS"] = "1"
# If GPU available, set to run on it
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


## Load Data

In [3]:
# Data download
df_x = pd.read_csv('/home/as6734/ml4fg_class_project/TCGA-PANCAN-HiSeq-801x20531/data.csv')
df_y = pd.read_csv('/home/as6734/ml4fg_class_project/TCGA-PANCAN-HiSeq-801x20531/labels.csv')

In [4]:
df_x.head()

,Unnamed: 0,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
1,sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
2,sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
3,sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
4,sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [5]:
df_y.head()

,Unnamed: 0,Class
0,sample_0,PRAD
1,sample_1,LUAD
2,sample_2,PRAD
3,sample_3,PRAD
4,sample_4,BRCA


In [6]:
# Clean data

# Drop first column that only notes sample number (which can be reconstructed from index if need be
df_x.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df_y.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# Remove columns with all 0.0 values
df_x = df_x.loc[:, (df_x != 0).any(axis=0)]

In [7]:
# Pre-process data

# Normalize values by mean
df_x=(df_x-df_x.mean())/df_x.std()
# Encode classes
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(df_y.values)
y = enc.transform(df_y.values)

In [8]:
df_x.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_6,gene_7,gene_8,gene_9,gene_10,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,-0.194678,-0.827513,0.159701,-1.947061,1.220812,-0.207838,0.180797,-0.125297,-0.065592,-0.082063,...,-1.299388,-0.921179,-0.877290,0.995625,-1.165344,0.389198,-0.869023,-1.187196,-0.116410,-0.261738
1,-0.194678,-2.013759,-1.414158,1.352264,-0.376283,-0.531890,-0.982474,-0.125297,-0.065592,-0.586397,...,-1.745985,-2.390720,-0.831373,0.591280,-2.548006,1.390759,0.623162,-0.342063,-1.655854,-0.261738
2,-0.194678,0.417087,1.156013,0.249651,0.112761,-0.391053,-0.092937,-0.125297,-0.065592,-0.586397,...,-1.033442,-1.059008,2.247399,0.232456,0.317682,-4.023107,-0.631986,0.886307,-1.854106,-0.261738
3,-0.194678,0.543549,1.325354,-0.098991,0.755269,0.395101,-0.127752,-0.125297,-0.065592,-0.586397,...,0.241148,0.044877,0.224815,1.718651,-0.263682,-0.521421,-0.793113,0.166070,-1.078268,-0.261738
4,-0.194678,-0.295770,-0.256947,-0.286234,-0.148750,-0.756645,-0.272995,-0.125297,-0.065592,-0.586397,...,0.133251,0.208122,0.837216,0.979312,0.196522,0.268824,-1.614832,-0.229734,-0.201463,-0.261738


In [9]:
y[0:5]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [10]:
# Split data, final percentages are approximately 64% train, 16% validation, 20% test
x_train, x_test, y_train, y_test = train_test_split(df_x.values, y, test_size=0.2, train_size=0.8, random_state=7, stratify=y)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, train_size=0.8, random_state=7, stratify=y_train)
dataset_sizes = {'train': len(x_train), 'validation': len(x_valid), 'test': len(x_test)}
print(dataset_sizes)

{'train': 512, 'validation': 128, 'test': 161}


In [20]:
# Convert to DataLoaders
x_train_to_tensor = torch.from_numpy(x_train).to(torch.float32).to(device)
y_train_to_tensor = torch.from_numpy(y_train).to(torch.float32).to(device)
x_valid_to_tensor = torch.from_numpy(x_valid).to(torch.float32).to(device)
y_valid_to_tensor = torch.from_numpy(y_valid).to(torch.float32).to(device)
x_test_to_tensor = torch.from_numpy(x_test).to(torch.float32).to(device)
y_test_to_tensor = torch.from_numpy(y_test).to(torch.float32).to(device)

# Second step: Creating TensorDataset for Dataloader
train_set = TensorDataset(x_train_to_tensor, y_train_to_tensor)
valid_set = TensorDataset(x_valid_to_tensor, y_valid_to_tensor)
test_set = TensorDataset(x_test_to_tensor, y_test_to_tensor)

# Create DataLoader
dataloaders = {
    'train': torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True),
    'validation': torch.utils.data.DataLoader(valid_set, batch_size=8),
    'test': torch.utils.data.DataLoader(test_set, shuffle=True, batch_size=8)
}

## Classical Architecture

In [12]:
# Classical Model Class
class ClassicalNet(nn.Module):
    # Initialize layers
    def __init__(self, num_feature):
        super().__init__()
        # Layers before quantum circuit
        self.layer_1 = nn.Linear(num_feature, 512)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.layer_2 = nn.Linear(512, 128)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.layer_3 = nn.Linear(128, 64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.layer_4 = nn.Linear(64, 5)
        self.c1 = nn.Conv1d(8, 8, 2)
        self.layer_5 = nn.Linear(4, 5)
        # Misc
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)

    # Forward pass procedure
    def forward(self, x):
        # First layer
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)

        # Second layer
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)

        # Third layer
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Quantum stand in layers 
        x = self.layer_4(x)
        x = self.c1(x)
        x = torch.flatten(x, 1)
        x = self.layer_5(x)
        return x

In [13]:
# Instantiate model
model = ClassicalNet(num_feature=len(df_x.columns))
# Make sure set to use GPU
model = model.to(device)
# Show model architecture summary
print(model)

ClassicalNet(
  (layer_1): Linear(in_features=20264, out_features=512, bias=True)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_4): Linear(in_features=64, out_features=5, bias=True)
  (c1): Conv1d(8, 8, kernel_size=(2,), stride=(1,))
  (layer_5): Linear(in_features=4, out_features=5, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)


## Train HQNN

In [14]:
step = 0.0004  # Initial learning rate
batch_size = 8
num_epochs = 10
gamma_lr_scheduler = 0.1  # Learning rate decay param

In [15]:
# Set up loss, optimizer, and learning rate decay manager
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=step)
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer, step_size=10, gamma=gamma_lr_scheduler
)

In [16]:
# Function to train model(s)
def train(model, loss_func, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000.0
    best_loss_train = 100000.0
    print('Training started:')
    for epoch in tqdm(range(num_epochs)):
        for phase in ['train', 'validation']:
            if phase == 'train':
                # Set model to train mode
                model.train()
            else:
                # Set model to eval mode
                model.eval()
            current_loss = 0.0
            n_batches = dataset_sizes[phase] // batch_size
            iter = 0
            for X, Y in dataloaders[phase]:
                batch_len = len(X)
                X = X.to(device)
                optimizer.zero_grad()  # Reset gradients
                # If in train mode, get loss, step optimizer
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(X)
                    _, preds = torch.max(outputs, 1)
                    loss = loss_func(outputs, Y)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                # Print iteration results
                current_loss += loss.item() * batch_len
                print('Phase: {} Epoch: {}/{} Iter: {}/{}'.format(phase, epoch+1, num_epochs, iter+1, n_batches+1),
                    end="\r",
                    flush=True)
                iter += 1

            # Get epoch stats and print
            epoch_loss = current_loss / dataset_sizes[phase]
            print('Phase: {} Epoch: {}/{} Loss: {:.4f}        '.format(
                    'train' if phase == 'train' else 'validation  ',
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                )
            )
            # Update best vars and make copy of best weights
            # if phase == "validation":
            #     best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "validation" and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss
            # Decay learning rate
            if phase == "train":
                scheduler.step()
    # Print final results
    model.load_state_dict(best_model_wts)
    print('Best test loss: {:.4f}'.format(best_loss))
    return model

In [17]:
model = train(model, loss_func, optimizer, exp_lr_scheduler, num_epochs=50)

Training started:


  0%|          | 0/50 [00:00<?, ?it/s]

Phase: train Epoch: 1/50 Loss: 1.6816        
Phase: validation   Epoch: 1/50 Loss: 1.6877        
Phase: train Epoch: 2/50 Loss: 1.6637        
Phase: validation   Epoch: 2/50 Loss: 1.6592        
Phase: train Epoch: 3/50 Loss: 1.6219        
Phase: validation   Epoch: 3/50 Loss: 1.6300        
Phase: train Epoch: 4/50 Loss: 1.6135        
Phase: validation   Epoch: 4/50 Loss: 1.6055        
Phase: train Epoch: 5/50 Loss: 1.5960        
Phase: validation   Epoch: 5/50 Loss: 1.5686        
Phase: train Epoch: 6/50 Loss: 1.5250        
Phase: validation   Epoch: 6/50 Loss: 1.5307        
Phase: train Epoch: 7/50 Loss: 1.4408        
Phase: validation   Epoch: 7/50 Loss: 1.4369        
Phase: train Epoch: 8/50 Loss: 1.3299        
Phase: validation   Epoch: 8/50 Loss: 1.2895        
Phase: train Epoch: 9/50 Loss: 1.2189        
Phase: validation   Epoch: 9/50 Loss: 1.1618        
Phase: train Epoch: 10/50 Loss: 1.0846        
Phase: validation   Epoch: 10/50 Loss: 0.9957        
Phase: t

In [18]:
# Save model weights locally in case of GCP crash
torch.save(model.state_dict(), './weights/classical_50epochs.pt')

## Test Classical

In [ ]:
# Load in saved weights, put in model, and than set to eval mode
best_weights = torch.load('./weights/classical_50epochs.pt')
model.load_state_dict(best_weights)
model.eval()

In [45]:
# Helper function to test model
def test(model, device, test_loader):
    preds = []
    targets = []
    with torch.no_grad():
        model.eval()
        for data, target in tqdm(test_loader):
            # Send the data and target to device
            data, target = data.to(device), target.to(device)
            if data.size()[0] < 8:
                break
            output = model(data)
            preds += enc.inverse_transform(output).flatten().tolist()
            targets += enc.inverse_transform(target).flatten().tolist()
    return preds, targets

preds, targets = test(model, 'cpu', dataloaders['test'])

  0%|          | 0/21 [00:00<?, ?it/s]

In [49]:
print(classification_report(y_true=targets, y_pred=preds))

              precision    recall  f1-score   support

        BRCA       0.79      0.87      0.83        60
        COAD       1.00      0.19      0.32        16
        KIRC       0.78      0.93      0.85        30
        LUAD       0.70      0.57      0.63        28
        PRAD       0.69      0.85      0.76        26

    accuracy                           0.76       160
   macro avg       0.79      0.68      0.68       160
weighted avg       0.77      0.76      0.73       160

